# About This Notebook
This notebook develops a procedure to request all Divvy station data via API and then save that data

<h3>Documentation</h3>
Live Station Data from Divvy Site
https://gbfs.divvybikes.com/gbfs/2.3/gbfs.json

# Import libraries

In [1]:
import pandas as pd
import requests
import datetime
import pytz

In [2]:
url = f"https://gbfs.lyft.com/gbfs/2.3/chi/en/station_status.json?"
response = requests.get(url)
data = response.json()
response

<Response [200]>

In [3]:
# get timestamp of dataset for filename
central_tz = pytz.timezone('America/Chicago')
status_timestamp = datetime.datetime.utcfromtimestamp(data['last_updated']).replace(tzinfo=pytz.utc)
status_timestamp = status_timestamp.astimezone(central_tz).strftime('%Y_%m_%d_%I%M%p')

# Prep Data
Too much data processing upstream could limit options downstream, but (for example) it's much easier to parse JSON fields prior to saving as CSV

In [4]:
#read into dataframe and rename columns
df_station_status= pd.DataFrame(data['data']['stations'])

In [5]:
# get timestamp for each station (which might be different from the dataset timestamp)
central_tz = pytz.timezone('America/Chicago')
df_station_status['timestamp'] = pd.to_datetime(df_station_status['last_reported'], unit='s', utc=True)
df_station_status['timestamp'] = df_station_status['timestamp'].dt.tz_convert(central_tz)
df_station_status['timestamp'] = df_station_status['timestamp'].dt.strftime('%m/%d/%Y %I:%M %p')

In [6]:
# pull out JSON fields
df_station_status['n_classic'] = df_station_status['vehicle_types_available'].apply(
    lambda x: next((item['count'] for item in x if item['vehicle_type_id'] == '1'), 0))
df_station_status['n_electric'] = df_station_status['vehicle_types_available'].apply(
    lambda x: next((item['count'] for item in x if item['vehicle_type_id'] == '2'), 0))
df_station_status['n_scooters'] = df_station_status['vehicle_types_available'].apply(
    lambda x: next((item['count'] for item in x if item['vehicle_type_id'] == '3'), 0))

In [7]:
#remove unneeded fields
df_station_status = df_station_status.drop(
    columns=['vehicle_docks_available','last_reported','vehicle_types_available'],axis=1)
df_station_status

,num_bikes_disabled,num_scooters_available,num_docks_available,num_docks_disabled,is_installed,is_renting,num_bikes_available,num_scooters_unavailable,num_ebikes_available,station_id,is_returning,timestamp,n_classic,n_electric,n_scooters
0,2,0.0,2,0,1,1,8,0.0,1,4843d153-c725-4230-a4c7-402da6a62143,1,11/16/2023 08:46 PM,7,1,0
1,2,0.0,22,0,1,1,7,0.0,0,a3a3e67b-a135-11e9-9cda-0a87ae2ba916,1,11/16/2023 08:46 PM,7,0,0
2,1,0.0,3,0,1,1,11,0.0,3,a3afd294-a135-11e9-9cda-0a87ae2ba916,1,11/16/2023 08:46 PM,8,3,0
3,0,1.0,0,0,1,1,14,0.0,6,a3a7f0ba-a135-11e9-9cda-0a87ae2ba916,1,11/16/2023 08:46 PM,8,6,1
4,0,0.0,8,0,1,1,7,0.0,1,a3b2c87c-a135-11e9-9cda-0a87ae2ba916,1,11/16/2023 08:46 PM,6,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1661,0,NaN,1,0,1,1,0,NaN,0,1715823822654071770,1,01/22/2023 09:45 PM,0,0,0
1662,0,0.0,7,0,1,1,2,0.0,2,1563698701206292480,1,01/22/2023 09:44 PM,0,0,0
1663,0,NaN,2,0,1,1,0,NaN,0,1674190591734328324,1,01/22/2023 09:46 PM,0,0,0
1664,0,NaN,4,0,1,1,0,NaN,0,1806749735787774444,1,05/04/2023 12:50 PM,0,0,0


# Create and Export CSV

In [8]:
#save data
filename = f"../data/station_status_{status_timestamp}.csv"
df_station_status.to_csv(filename, index=False)